In [1]:
import sqlite3
import pandas as pd

In [2]:
from sqlite3.dbapi2 import Cursor
connection = sqlite3.connect('library.db')
Cursor = connection.cursor()

In [3]:
# get the names of All tabels in resturant database 
query = 'SELECT name FROM  sqlite_master  WHERE  type = "table" AND  name NOT LIKE "sqlite_%";'
df = pd.read_sql_query( query, connection)
df 

,name
0,Patrons
1,Books
2,Loans


In [4]:
# explore coumns in Books table
query = 'SELECT * FROM  Books'
df = pd.read_sql_query( query, connection)
df.head(2)

,BookID,Title,Author,Published,Barcode
0,1,"Walden, and On The Duty Of Civil Disobedience",Henry David Thoreau,1849,3100471153
1,2,The Scarlet Letter,Nathaniel Hawthorne,1850,4589408958


In [5]:
# explore Patrons table
query = 'SELECT * FROM  Patrons'
df = pd.read_sql_query( query, connection)
df.head(2)

,FirstName,LastName,Email,PatronID
0,Maggi,Domney,mdomney0@wisdompets.com,1
1,Javier,Dawks,jdawks1@red30design.com,2


In [6]:
# explore Loans Table 
query = 'SELECT * FROM  Loans'
df = pd.read_sql_query( query, connection)
df.head(2)

,LoanID,BookID,PatronID,LoanDate,DueDate,ReturnedDate
0,1,165,49,2016-06-01,2016-06-15,2016-06-15
1,2,123,100,2016-06-01,2016-06-15,2016-06-15


# Challenge 1

find the available items of Dracula book 


In [7]:
# total number of Dracula in library  
query = 'SELECT count(Title) totalcopiesDracula FROM  Books where Title = "Dracula" ' 
df = pd.read_sql_query( query, connection)
df.head(2)

,totalcopiesDracula
0,3


In [8]:
# total number of loaned Dracula book 
query = ' SELECT (SELECT count(Title) FROM  Books where Title = "Dracula") - \
(SELECT count(b.Title) FROM  Books b join Loans l ON b.BookID = l.BookID where Title = "Dracula" AND l.ReturnedDate  IS Null ) AS Availablebooks' 
df = pd.read_sql_query( query, connection)
df.head(2)

,Availablebooks
0,2


In [9]:
# total number of available books from Dracula 
query = 'SELECT count(b.Title) totalbrowwedDracula FROM  Books b join Loans l  ON b.BookID = l.BookID where Title = "Dracula" AND l.ReturnedDate  IS Null  ' 
df = pd.read_sql_query( query, connection)
df.head(2)

,totalbrowwedDracula
0,1


# challenge 2 

insert new copies from already books in library with different barcodes

In [10]:
Cursor.executescript("""
    INSERT INTO Books  
    (Title , Author, Published ,  Barcode)
    VALUES
    ("Dracula" , "Bram Stoker" , "1897" , "481927482") , 
    ("Gulliver's Travels" , "Jonathan Swift" , "1729" , "4899254401");
    """)

In [11]:
query = 'SELECT * From Books ORDER by BookID Desc ' 
df = pd.read_sql_query( query, connection)
df.head(2)

,BookID,Title,Author,Published,Barcode
0,202,Gulliver's Travels,Jonathan Swift,1729,4899254401
1,201,Dracula,Bram Stoker,1897,481927482


# challenge 3 

Insert new book in loans table with details about the date of loaning and Due date

In [12]:
Cursor.executescript("""
    INSERT INTO Loans  
    (BookID , PatronID , LoanDate , DueDate)
    VALUES
    ((SELECT BookID From books WHERE Barcode = "2855934983") , 
    (SELECT PatronID From Patrons Where Email = "jvaan@wisdompets.com" ) , "2020-08-25" , "2020-09-08");
    """)

# challenge 4

check the people who should return their books back to librarz on 17 July 2020 


In [13]:
query = 'SELECT p.FirstName	 , p.LastName	, p.Email  ,  b.title  , l.DueDate From Loans l JOIN Books b ON  l.BookID = b.BookID \
JOIN Patrons p ON  l.PatronID = p.PatronID   where l.DueDate = "2020-07-13" AND l.ReturnedDate IS Null ' 
df = pd.read_sql_query( query, connection)
df

,FirstName,LastName,Email,Title,DueDate
0,Kala,Simonian,ksimonian2q@landonhotel.com,The Count of Monte Cristo,2020-07-13
1,Lydie,Awmack,lawmack1a@samoca.org,The Prophet,2020-07-13
2,Anthe,Dinjes,adinjesh@samoca.org,Second Treatise of Government,2020-07-13
3,Cornelia,Koppe,ckoppe1m@landonhotel.com,Dracula,2020-07-13


# challenge 5

change the returned date for a book  

In [14]:
Cursor.executescript("""UPDATE Loans
SET ReturnedDate = "2020-07-05"
WHERE  BookID = (Select BookID from Books where Barcode = "6435968624"); """)

In [15]:
Cursor.executescript("""UPDATE Loans
SET ReturnedDate = "2020-07-05"
WHERE  BookID = (Select BookID from Books where Barcode = "5677520613"); """)

#challenge 6

The number of loaning for each person 

In [16]:
query = 'SELECT p.FirstName	 , p.LastName	, p.Email , count(l.LoanDate) totalcheck  From Patrons p Join Loans l ON p.PatronID = l.PatronID \
GROUP BY l.PatronID Order BY totalcheck  limit 10 ' 
df = pd.read_sql_query( query, connection)
df

,FirstName,LastName,Email,totalcheck
0,Alaster,Ruggles,aruggles1w@red30design.com,9
1,Caril,Matejic,cmatejic2a@orangevalleycaa.org,9
2,Merle,Sukbhans,msukbhans20@orangevalleycaa.org,11
3,Norby,Sleight,nsleight1r@kinetecoinc.com,12
4,Curr,Fittall,cfittall2l@orangevalleycaa.org,12
5,Aleen,Fasey,afasey2@kinetecoinc.com,14
6,Cammi,Kynett,ckynett6@orangevalleycaa.org,14
7,Orton,Stavers,ostaversb@orangevalleycaa.org,14
8,Carla,Swinfen,cswinfenf@kinetecoinc.com,14
9,Jack,Vaan,jvaan@wisdompets.com,14


# challenge 7

In [17]:
query = 'SELECT b.title , b.Author , b.BookID , b.Published FROM  Books b Join Loans l ON b.BookID = l.BookID where b.Published > 1890 AND b.Published < 1900 \
AND l.ReturnedDate IS NOT null group by b.BookID ORdER BY b.title '
df = pd.read_sql_query( query, connection)
df.head(2)

,Title,Author,BookID,Published
0,Also sprach Zarathustra,Friedrich Wilhelm Nietzsche,70,1896
1,Also sprach Zarathustra,Friedrich Wilhelm Nietzsche,95,1896


In [18]:
len(df)

35

# challenge 8

How many books published every year 

In [19]:
query = 'SELECT b.Published , count(DISTINCT(b.title)) totalbooks  FROM  Books b  Group by b.Published order by totalbooks DESC  '
df = pd.read_sql_query( query, connection)
df.head(6)

,Published,totalbooks
0,1890,3
1,1895,3
2,1729,2
3,1847,2
4,1850,2
5,1859,2


# challenge 9

The most Popular checked out books 

In [20]:
query = 'SELECT b.Title , count(l.LoanID) totalchecked  FROM  loans l JOIN Books b ON b.BookID = l.BookID Group by b.Title order by totalchecked DESC  '
df = pd.read_sql_query( query, connection)
df.head(6)

,Title,totalchecked
0,"Frankenstein; Or, The Modern Prometheus",53
1,"The Awakening, and Selected Short Stories",43
2,The War of the Worlds,43
3,The Scarlet Letter,42
4,The Picture of Dorian Gray,40
5,Alice's Adventures in Wonderland,38
